#Next Match

Find the next qualification match for 2158 that has not been played, and give some useful information



In [22]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

from common import canalytics

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Next_Match_Analysis')

tba_matches_by_team = pd.read_csv('./data/tba_matches_by_team.csv', index_col=0)
team_summary = pd.read_csv('./data/team_summary.csv', index_col=0)
tba_matches_by_team

Loading BokehJS ...

,match,post_result_time,predicted_time,team,climb,fouls,techfouls,alliance
0,1,NaN,2022-04-01 09:15,4336,None,0,0,blue
0,1,NaN,2022-04-01 09:15,5863,None,0,0,blue
0,1,NaN,2022-04-01 09:15,2080,None,0,0,blue
0,1,NaN,2022-04-01 09:15,6068,None,0,0,red
0,1,NaN,2022-04-01 09:15,5872,None,0,0,red
...,...,...,...,...,...,...,...,...
73,76,NaN,2022-04-02 11:48,179,None,0,0,blue
73,76,NaN,2022-04-02 11:48,7834,None,0,0,blue
73,76,NaN,2022-04-02 11:48,8118,None,0,0,red
73,76,NaN,2022-04-02 11:48,8418,None,0,0,red


In [25]:
# Determine which match is next for 2158, and their alliances, and opponents
# TODO: Check to make sure this works.


In [28]:
next_match = min(tba_matches_by_team[ ( tba_matches_by_team['post_result_time'].isna() ) & (tba_matches_by_team['team']==2190)].match)
next_match

7

In [29]:
# For each member:
alliances = tba_matches_by_team[ tba_matches_by_team['match'] == next_match ]
alliances = alliances.merge(team_summary,on="team",how='left')
alliances.sort_values(['alliance','team'],inplace=True)



In [30]:
alliances

,match,post_result_time,predicted_time,team,climb,fouls,techfouls,alliance,matches,scouted_matches,...,total_cargo_avg,auto_cargo_avg,teleop_cargo_avg,climb_points_avg,highest_endgame_position,oprs,pre_oprs,pre_climb_points,pre_district_rank,pre_district_points
0,7,NaN,2022-04-01 10:03,2190,None,0,0,blue,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,2022-04-01 10:03,5965,None,0,0,blue,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,NaN,2022-04-01 10:03,7094,None,0,0,blue,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,NaN,2022-04-01 10:03,233,None,0,0,red,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,NaN,2022-04-01 10:03,1818,None,0,0,red,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,NaN,2022-04-01 10:03,8418,None,0,0,red,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# - OPRs
# - Average Cargo 
# For Alliance
# - Average CLimb points
# - previous OPR ( by flag )
# - previous District Info    
alliances=alliances.round(decimals=2)
alliances=alliances.fillna("")

alliance_summary=alliances.groupby('alliance').agg(
                    avg_climb_points = pd.NamedAgg(column='climb_points_avg',aggfunc='sum'),
                    avg_cargo = pd.NamedAgg(column='total_cargo_avg',aggfunc='sum')
)


source = ColumnDataSource(alliances)


columns = [
        TableColumn(field="alliance", title="Alliance"),
        TableColumn(field="team", title="Team"),
        TableColumn(field="oprs", title="OPRS (Event)"),
        TableColumn(field="pre_oprs", title="OPRS (Prior)"),
        TableColumn(field="total_cargo_avg", title="Total Cargo"),
        TableColumn(field="auto_cargo_avg", title="Auto Cargo"),
        TableColumn(field="teleop_cargo_avg", title="Teleop Cargo"),
        TableColumn(field="highest_endgame_position", title="Highest"),
        TableColumn(field="climb_points_avg", title="Climb Pts"),
        TableColumn(field="pre_climb_points", title="Prior Climb Pts")]

next_match_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        fit_columns=True,
        reorderable=True)

next_match

7

In [32]:
alliance_summary

,avg_climb_points,avg_cargo
alliance,,
blue,,
red,,


In [33]:
show(next_match_table)